In [5]:
import cv2
import os
import time

# ---------------------------------------------------------
# CONFIGURATION
# ---------------------------------------------------------
SAVE_DIR = r"C:\Users\12272\527project\data\data\auto"
TARGET_SIZE = (100, 100)
CAPTURE_DURATION = 10  # How long to run the camera (in seconds)

# Ensure directory exists
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)
    print(f"[INFO] Created directory: {SAVE_DIR}")

# ---------------------------------------------------------
# 1. INITIALIZE FACE DETECTOR
# ---------------------------------------------------------
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
if face_cascade.empty():
    raise IOError("Error loading Haar Cascade XML file.")

# ---------------------------------------------------------
# 2. INITIALIZE CAMERA
# ---------------------------------------------------------
# Updated to Index 0 based on your feedback
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

if not cap.isOpened():
    raise RuntimeError("Could not open webcam.")

print(f"[INFO] Camera opened. Auto-capture will run for {CAPTURE_DURATION} seconds.")
# print("[INFO] Press 'q' to quit early.")

# ---------------------------------------------------------
# 3. MAIN LOOP (TIMER + AUTO SAVE)
# ---------------------------------------------------------
start_time = time.time()
img_count = 0

while True:
    # Check if time is up
    elapsed_time = time.time() - start_time
    if elapsed_time > CAPTURE_DURATION:
        print("[INFO] Time limit reached.")
        break

    ret, frame = cap.read()
    if not ret:
        print("[ERROR] Failed to read frame")
        break

    # Convert to grayscale for detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)

    # -----------------------------------------------------
    # AUTO-SAVE LOGIC
    # -----------------------------------------------------
    if len(faces) > 0:
        # Find the largest face (Area = w * h)
        (x, y, w, h) = max(faces, key=lambda r: r[2] * r[3])
        
        # Crop the face
        face_roi = gray[y:y+h, x:x+w]
        
        # Resize to 100x100
        final_image = cv2.resize(face_roi, TARGET_SIZE, interpolation=cv2.INTER_AREA)
        # ==========================================
        # 新增：模糊检测 (Blur Detection)
        # ==========================================
        # 计算拉普拉斯方差
        variance = cv2.Laplacian(final_image, cv2.CV_64F).var()
        
        # 阈值 (Threshold): 这个值需要根据你的摄像头调整
        # 通常 100 是一个通用的基准。如果太严导致存不下图，就调低到 50 或 60
        if variance < 80: 
            print(f"[SKIP] Image too blurry (Var: {variance:.2f})")
            # 在画面上画红框提示模糊
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
            cv2.putText(frame, "BLURRY", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            
            # 显示画面但不保存
            cv2.imshow("Auto Face Collector", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'): break
            continue  # 跳过本次循环，不执行下面的保存代码
        # Save to disk
        # We use time.time() * 1000 to get milliseconds so names are unique
        # or simply use the img_count variable
        filename = f"face_{int(time.time())}_{img_count}.png"
        save_path = os.path.join(SAVE_DIR, filename)
        
        cv2.imwrite(save_path, final_image)
        img_count += 1
        
        # visual feedback on the console (optional, might spam)
        # print(f"[SAVED] {filename}")

        # Draw a GREEN rectangle on the preview to show it's saving
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, "RECORDING", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # -----------------------------------------------------
    # DISPLAY INFO
    # -----------------------------------------------------
    # Show timer on screen
    remaining = int(CAPTURE_DURATION - elapsed_time)
    cv2.putText(frame, f"Time Left: {remaining}s", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.putText(frame, f"Images Saved: {img_count}", (10, 60), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    cv2.imshow("Auto Face Collector", frame)

    # Allow quitting early with 'q'
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        print("[INFO] User quit early.")
        break

# ---------------------------------------------------------
# CLEANUP
# ---------------------------------------------------------
cap.release()
cv2.destroyAllWindows()
print(f"[INFO] Finished. Total images saved: {img_count}")

[INFO] Camera opened. Auto-capture will run for 10 seconds.
[SKIP] Image too blurry (Var: 42.38)
[SKIP] Image too blurry (Var: 7.09)
[SKIP] Image too blurry (Var: 40.76)
[SKIP] Image too blurry (Var: 37.61)
[INFO] Time limit reached.
[INFO] Finished. Total images saved: 182
